# Titanic

In [10]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# To reduce output size while working with vscode
%config InlineBackend.figure_format = 'png'

%matplotlib inline

# Display all columns
pd.options.display.max_columns = None

FIGURES_PATH = "plots/"
DATASETS_PATH = "datasets/"

def save_fig(name, extension="png", resolution=300):
    os.makedirs(FIGURES_PATH, exist_ok=True)
    path = os.path.join(FIGURES_PATH, name + "." + extension)
    # print("Saving figure", name)
    plt.tight_layout()
    plt.savefig(path, format=extension, dpi=resolution)

np.random.seed(42)

In [11]:
TRAIN_PATH = os.path.join(DATASETS_PATH, "train.csv")
TEST_PATH = os.path.join(DATASETS_PATH, "test.csv")

titanic = pd.read_csv(TRAIN_PATH, index_col=0)
titanic_test = pd.read_csv(TEST_PATH, index_col=0)

In [12]:
titanic

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [13]:
titanic.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [14]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [15]:
corr = titanic.corr()
corr["Survived"].abs().sort_values()

SibSp       0.035322
Age         0.077221
Parch       0.081629
Fare        0.257307
Pclass      0.338481
Survived    1.000000
Name: Survived, dtype: float64

In [ ]:
for attr in ["Age", "Fare"]:
    plt.subplots(figsize=(8, 6))
    sns.distplot(titanic[attr], kde=False, rug=True)
    save_fig(attr)

for attr in ["Survived", "Pclass", "Sex", "SibSp", "Parch", "Embarked"]:
    plt.subplots(figsize=(8, 6))
    sns.countplot(attr, data=titanic)
    save_fig(attr)

In [16]:
num_attribs = ["Age", "Fare"]
cat_attribs = ["Pclass", "Sex", "SibSp", "Parch", "Embarked"]

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import sklearn.preprocessing as preprocessing
import sklearn.impute as impute

num_pipeline = Pipeline([
    ("imputer", impute.SimpleImputer(strategy="median")),
    ("scaler", preprocessing.StandardScaler()),
])

cat_pipeline = Pipeline([
    ("imputer", impute.SimpleImputer(strategy="most_frequent")),
    ("ecoder", preprocessing.OneHotEncoder())
])

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs)
])